In [1]:
import json
from collections import Counter
import jieba
import jieba.posseg as pseg
import random
import numpy as np

In [2]:
def read_questions_from_file(file_path):
    questions = []
    with open(file_path, 'r') as f:
        for line in f:
            question = json.loads(line)
            questions.append(question)
    return questions

In [3]:
file_path = 'test_questions.jsonl'
questions = read_questions_from_file(file_path)
for question in questions[:3]:
    print(question)

{'id': 0, 'question': '能否根据2020年金宇生物技术股份有限公司的年报，给我简要介绍一下报告期内公司的社会责任工作情况？'}
{'id': 1, 'question': '请根据江化微2019年的年报，简要介绍报告期内公司主要销售客户的客户集中度情况，并结合同行业情况进行分析。'}
{'id': 2, 'question': '2019年四方科技电子信箱是什么?'}


In [4]:
def count_noun_frequency(questions):
    counter = Counter()
    for question in questions:
        # 使用jieba进行分词和词性标注
        tokens = pseg.cut(question['question'])
        # 只保留名词
        nouns = [word for word, flag in tokens if flag.startswith('n')]
        counter.update(nouns)
    return counter

In [5]:
def keyword_in_questions(keyword):
    id_List = []
    for question in questions:
        if keyword not in question['question']:
            continue
        id_List.append(question['id'])
    return len(id_List), id_List

In [6]:
def keywords_not_in_questions(keywords):
    id_List = []
    for question in questions:
        f = False
        for keyword in keywords:
            if keyword not in question['question']:
                continue
            f = True
            break
        if f == False:
            id_List.append(question['id'])
    return len(id_List), id_List

# 名词统计

In [7]:
counter = count_noun_frequency(questions)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/gd/qn8pw1px5v91wttpc6zxzmdw0000gn/T/jieba.cache
Loading model cost 0.693 seconds.
Prefix dict has been built successfully.


In [8]:
print(counter.most_common(20))

[('有限公司', 2382), ('股份', 1983), ('小数', 1519), ('营业', 871), ('增长率', 752), ('科技股份', 454), ('现金', 308), ('比率', 302), ('科技', 282), ('集团股份', 279), ('负债', 269), ('费用', 259), ('公司', 255), ('企业', 255), ('年报', 249), ('利润', 203), ('情况', 197), ('人数', 195), ('职工', 191), ('收益', 185)]


In [9]:
x = [e for e in  counter.most_common(2000) if len(e[0])>=4]


In [10]:
x[:10]

[('有限公司', 2382),
 ('科技股份', 454),
 ('集团股份', 279),
 ('金融资产', 164),
 ('财务费用', 154),
 ('管理费用', 154),
 ('销售费用', 154),
 ('投资收益', 151),
 ('法定代表', 120),
 ('利息收入', 101)]

# 问题分类 by 关键词1

In [11]:
ks = ["是多少","为多少", "是什么", "是否相同", "是谁", "请提供"]
es = {k: keyword_in_questions(k) for k in ks}

In [12]:
for k, v in es.items():
    print(k, v[0])

是多少 3396
为多少 324
是什么 573
是否相同 47
是谁 21
请提供 129


In [13]:
n, id_List = keywords_not_in_questions(ks)
"达到了多少" # 5
"有多少" # 10
"请计算" # 7
n

524

In [14]:
for i in random.sample(id_List, 10):
    print(i, questions[i])

4654 {'id': 4654, 'question': '长期股权投资的价值变动如何影响公司的净资产和盈利能力？'}
790 {'id': 790, 'question': '什么是总资产周转率？'}
1613 {'id': 1613, 'question': '如何分析营业收入的变动趋势？'}
728 {'id': 728, 'question': '什么是归属于母公司所有者权益？'}
1294 {'id': 1294, 'question': '根据2019年横店集团得邦照明股份有限公司的年报，请简要介绍报告期内公司供应商集中度如何？请结合同行业情况简要分析。'}
4387 {'id': 4387, 'question': '什么是应付账款？'}
4861 {'id': 4861, 'question': '根据2019年红宝丽的年报，请简要介绍报告期内公司主要销售客户客户集中度如何？请结合同行业情况简要分析。'}
2038 {'id': 2038, 'question': '请简要分析广东奥飞数据科技股份有限公司2019年关键审计事项的情况。'}
1862 {'id': 1862, 'question': '什么是长期借款？'}
343 {'id': 343, 'question': '什么是财务风险？'}


In [15]:
nested_list = [v[1] for _, v in es.items()]
sum([v[0] for _, v in es.items()]), len(set([item for sublist in nested_list for item in sublist])), n

(4490, 4476, 524)

In [35]:
ks = ["什么是", "分析", "影响", "评估", "意思", "为什么"] # 主观题：分析和百科
es = {k: keyword_in_questions(k) for k in ks}

In [36]:
nested_list = [v[1] for _, v in es.items()]
sum([v[0] for _, v in es.items()]), len(set([item for sublist in nested_list for item in sublist]))

(371, 367)

# 问题分类 by 关键词2

In [47]:
ks = [
    "销售费用", "流动比率", "三费比重", "现金及现金等价物", "利息收入", "综合收益总额", "财务费用率",
    "货币资金增长率", "营业利润率", "研发经费与营业收入比值", "公司网址", "总负债增长率", "证券代码", "所得税费用",
    "法定代表人", "应收款项融资", "管理费", "流动负债", "固定资产", "职工薪酬", "投资收益", "净利润", "营业成本",
    "营业税金及附加", "研发人员", "职工人数", "无形资产", "衍生金融资产", "博士及以上人数", "企业名称",
    "收回投资收到的现金", "净利润增长率", "研发费用", "研发经费", "办公地址", "速动比率", "公允价值变动收益",
    "技术人员", "利润总额", "无形资产增长率", "每股经营现金流量", "职工总数", "电子信箱", "环境信息", "外文名称",
    "社会责任", "核心竞争力", "重大关联交易", "硕士及以上人员", "利息支出", "毛利率", "证券简称", "净资产",
    "资产负债比率",
]

es = {k: keyword_in_questions(k) for k in ks}

In [48]:
nested_list = [v[1] for _, v in es.items()]
sum([v[0] for _, v in es.items()]), len(set([item for sublist in nested_list for item in sublist]))

(4095, 3662)

In [26]:
for k, v in es.items():
    print(k, v[0])

销售费用 154
流动比率 51
三费比重 49
现金及现金等价物 97
利息收入 101
综合收益总额 76
财务费用率 50
货币资金增长率 50
营业利润率 50
研发经费与营业收入比值 33
公司网址 47
总负债增长率 50
证券代码 50
所得税费用 54
法定代表人 120
应收款项融资 52
管理费 204
流动负债 152
固定资产 157
职工薪酬 42
投资收益 151
净利润 208
营业成本 151
营业税金及附加 51
研发人员 101
职工人数 98
无形资产 156
衍生金融资产 65
博士及以上人数 42
企业名称 49
收回投资收到的现金 36
净利润增长率 49
研发费用 153
研发经费 150
办公地址 49
速动比率 52
公允价值变动收益 54


数据分析简单结论：数据库的合理构建与信息抽取的准确能是我们至少能cover 初赛里80%的题目

# 一、系统设计

我们的问答系统需要完成以下几个关键任务：

数据预处理：对70G的上市公司财报数据进行清洗、预处理和结构化存储。

实体识别与关系抽取：对用户查询中的实体和关系进行识别并抽取。

意图识别与问题分类：根据用户查询的上下文理解用户意图并将问题分类。

数据查询与分析：根据用户查询从知识图谱中检索信息，对检索到的数据进行统计和分析。

回答生成：对于开放性问题，使用ChatGLM2-6B模型生成回答。

# 二、技术栈

深度学习模型：ChatGLM2-6B

自然语言处理工具：Spacy、GPT4

知识图谱工具或者传统数据库：Neo4j、Mysql

向量数据库：Faiss、Milvus

数据处理和分析工具：Pandas、Numpy、Scipy、Apache Spark

其他工具：Docker/Kubernetes（环境隔离和部署），Git（版本控制和协作）

# 三、技术方案

总体思路：数据库(知识图谱)作为核心查询工具，向量数据库负责兜底，大模型润色。直至前面的工作效果比较好之后，再考虑模型微调。

敏捷开发，先把系统搭建起来，再逐步优化每一个模块。

1. 数据预处理和分析：完善数据分析，问题归类，关键词整理。
2. 构建向量数据库：按照句子切割的方式，构建向量数据库（这个实现难度最低）。
3. prompt工程：直接与语言模型交互的prompt处理相关的细节
4. 信息抽取：抽取pdf信息, 财报的结构大体是相同的 ,我倾向于先在1到2个财报上做实验。
5. 知识图谱构建依据抽取的构建数据库（知识图谱）
6. 规则引擎：引入tool(LangChain)构建推理的规则（知识图谱）
7. 常识数据库：构建一个金融常识的数据库，优先cover初赛里出现的问题
8. 代码重构和优化：工程性代码，整理一下开发中的冗余代码
9. 模型微调：考虑模型微调，提高开放型问题的回答质量


# 四、时间规划

初赛开放评测时间为7月24日—8月16日，系统每天提供3次提交机会。本赛段最后一次评测截止时间为8月16日18:00。


Sprint 1 - 数据预处理和向量数据库（7月24日-7月27日，4天）：在这个Sprint中，你可以先分析数据并构建出基础的向量数据库。

Sprint 2 - 信息抽取和知识图谱构建（7月28日-7月31日，4天）：在这个Sprint中，你可以开始进行信息抽取，并尝试构建出基础的知识图谱。

Sprint 3 - Prompt（8月1日--8月3日，3天）：在这个Sprint中，你设计优化prompt。

Sprint 4 - 规则引擎和常识数据库（8月4日-8月6日，3天）：在这个Sprint中，你可以进一步完善你的知识图谱，并开始构建规则引擎和常识数据库。

Sprint 5 - 代码优化和系统集成（8月7日-8月9日，3天）：这个Sprint可以用来进行代码的优化和系统的集成。

Sprint 6 - 测试和调整（8月10日-8月12日，3天）：在最后一个Sprint中，你可以进行系统的全面测试，找出并修复可能存在的问题，并对系统进行最后的优化。

Buffer（8月13日-8月16日，4天）：这个时间段可以作为缓冲期，用来处理在前面Sprints中没有预见到的问题，或者进行一些额外的优化。


明日重点：

    1. 考虑一下自己对哪一部分感兴趣，希望负责哪一部分工作
    2. 看一下胡骁的框架，简单跑一下，提提优化意见或者是否有必要重构
    3. 独自做一下数据分析，一部分是对问题的分析，一部分是对财报的分析

下一次会议：

    1. 5分钟，讨论分工。
    2. 工程框架的优化方向。
    3. 数据分析的讨论。